In [ ]:
#function terrainCorrection(image) {

imgGeom = image.geometry()
srtm = ee.Image('USGS/SRTMGL1_003').clip(imgGeom) // 30m srtm 
sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

#Article ( numbers relate to chapters)
#2.1.1 Radar geometry 
theta_i = image.select('angle')
phi_i = ee.Terrain.aspect(theta_i)
    .reduceRegion(ee.Reducer.mean(), theta_i.get('system:footprint'), 1000)
    .get('aspect')

# 2.1.2 Terrain geometry
alpha_s = ee.Terrain.slope(srtm).select('slope')
phi_s = ee.Terrain.aspect(srtm).select('aspect')

# 2.1.3 Model geometry
# reduce to 3 angle
phi_r = ee.Image.constant(phi_i).subtract(phi_s)

# convert all to radians
phi_rRad = phi_r.multiply(Math.PI / 180)
alpha_sRad = alpha_s.multiply(Math.PI / 180)
theta_iRad = theta_i.multiply(Math.PI / 180)
ninetyRad = ee.Image.constant(90).multiply(Math.PI / 180)

# slope steepness in range (eq. 2)
alpha_r = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

# slope steepness in azimuth (eq 3)
alpha_az = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

# local incidence angle (eq. 4)
theta_lia = (alpha_az.cos().multiply((theta_iRad.subtract(alpha_r)).cos())).acos()
theta_liaDeg = theta_lia.multiply(180 / Math.PI)